# Genshin Calculator:

In [1]:
# =================================  Next Goal of Development =================================
# 1. 使用excel导入角色数据1(complete)
##        Use dilimeter to seperate character's info
##        1.2 怎么计算解决状态类大招问题（eg. Xiao, Itto etc.） 
# 2. 导入怪物数据(ongoing)
# 3. 

In [2]:
import pandas as pd

## Class

In [3]:
# character class
# Read in using specific format and calculate automatically
class Character:
    def __init__(self, name, level, skill_scale, vision_element=None, damage_attribute=None, survival_attribute=None):
        self.name = name
        self.level = level       
        self.vision_element = vision_element
        self.damage_attribute = damage_attribute                    
        self.survival_attribute = survival_attribute 
        self.skill_scale = skill_scale
            
    def expect_dmg(self, monster = None):
        if monster == None:
            monster_level = 93
            monster_res = 0.1
            print(f'<No monster type specified, using default monster level and resistance{93, 0.1}>')
        else: 
            monster_level = monster.level
            monster_res = monster.resistence
        #calculation:
        e_dmg, q_dmg = self.calculate_dmg(monster_level, monster_res)
        #construct dataframe:
        dmg_data = {'normal_e': [e_dmg[0]], 'crit_e': [e_dmg[1]], 'normal_q': [q_dmg[0]], 'crit_q': [q_dmg[1]]}
        dmg_df = pd.DataFrame(data=dmg_data)
        # return a panda dataframe of dmg:
        return dmg_df
    
    # helper function to calculate dmg and return a list
    def calculate_dmg(self, monster_level, monster_res):
        e_dmg_list = []
        q_dmg_list = []
        #calculate dmg:
        e_dmg = (self.skill_scale.e/100)*self.damage_attribute.atk*(1+self.damage_attribute.dmg_bonus/100)*((self.level + 100)/(self.level+monster_level+200))*(1-monster_res)
        e_crit_dmg = e_dmg * (1+self.damage_attribute.crit_damage/100)
        q_dmg = (self.skill_scale.q/100)*self.damage_attribute.atk*(1+self.damage_attribute.dmg_bonus/100)*((self.level + 100)/(self.level+monster_level+200))*(1-monster_res)
        q_crit_dmg = q_dmg * (1+self.damage_attribute.crit_damage/100)
        #append the list:
        e_dmg_list.append(round(e_dmg,2))
        e_dmg_list.append(round(e_crit_dmg,2))
        q_dmg_list.append(round(q_dmg,2))
        q_dmg_list.append(round(q_crit_dmg,2))
        #print to console:
        print(f'===== expected e dmg =====')
        print(f'normal e: {e_dmg:10.2f}')
        print(f'crit e:   {e_crit_dmg:10.2f}')
        print(f'===== ============ =====')
        print(f'===== expected q dmg =====')
        print(f'normal q: {q_dmg:10.2f}')
        print(f'crit q:   {q_crit_dmg:10.2f}')
        print(f'===== ============ =====')
        return e_dmg_list, q_dmg_list
    
    def show(self):
        print(f'Character name:   {self.name}')
        print(f'Character level:  {self.level}')
        print(f'Character vision: {self.vision_element}')
        self.damage_attribute.show()
        self.skill_scale.show()

In [4]:
# monster class
# Calculate the damage on specific monster type
class Monster:
     def __init__(self, name, level, element, resistence):
        self.name = name
        self.level = level       
        self.element = element
        self.resistence = resistence         
        

In [5]:
class Damage_attribute:
    def __init__(self, atk, elemental_mastery, crit_rate, crit_damage, cd_reduction, dmg_bonus):
        self.atk = atk
        self.elemental_mastery = elemental_mastery
        self.crit_rate = crit_rate                      # display in percent
        self.crit_damage = crit_damage                  # display in percent
        self.cd_reduction = cd_reduction
        self.dmg_bonus = dmg_bonus                      # display in percent

    
    def show(self):                              # print method
        print(f'damage attribute: ')
        print(f'  atk:               {self.atk}')
        print(f'  elemental_mastery: {self.elemental_mastery}')
        print(f'  crit_rate:         {self.crit_rate}%')
        print(f'  crit_damage:       {self.crit_damage}%')
        print(f'  cd_reduction:      {self.cd_reduction}')
        print(f'  dmg_bonus:         {self.dmg_bonus}%')   

In [6]:
# skill class
# record the scaling of the character's skill
class Skill_scaling:
    def __init__(self, e, q):
        self.e = e
        self.q = q
        
    def show(self):
        print(f'character skill e scale: {self.e}%')
        print(f'character skill q scale: {self.q}%')

In [7]:
# read in the character info
# name, level, skill_scale, vision_element=None, damage_attribute=None, survival_attribute=None)


def read_character(path):
    char_list = []
    df = pd.read_csv(path)
    for index, row in df.iterrows():
        dmg_att = Damage_attribute(row['atk'], row['elemental_mastery'], row['crit_rate'], row['crit_damage'], row['cd_reduction'], row['dmg_bonus'])
        skill = Skill_scaling(row['e'], row['q'])
        char = Character(row['name'], row['level'], skill, row['vision'], dmg_att)
        char_list.append(char)
    return char_list

## Calculation

## First Read in the Character Info

#### Just a demo

In [12]:
#定好靶子
ice_tree = Monster('IceTree', 93, 'cryo', 0.1)

In [9]:
#read in character info
chars = read_character('data.csv')

In [13]:
#print char info
chars[0].show()

chars[1].show()

Character name:   Xiao
Character level:  90
Character vision: Anemo
damage attribute: 
  atk:               2536
  elemental_mastery: 117
  crit_rate:         66.4%
  crit_damage:       171.3%
  cd_reduction:      0
  dmg_bonus:         46.6%
character skill e scale: 404.0%
character skill q scale: 0.0%
Character name:   Childe
Character level:  90
Character vision: Hydro
damage attribute: 
  atk:               1930
  elemental_mastery: 40
  crit_rate:         75.4%
  crit_damage:       130.1%
  cd_reduction:      0
  dmg_bonus:         90.3%
character skill e scale: 129.6%
character skill q scale: 835.2%


In [14]:
chars[0].expect_dmg(ice_tree)

===== expected e dmg =====
normal e:    6705.97
crit e:     18193.31
===== ============ =====
===== expected q dmg =====
normal q:       0.00
crit q:         0.00
===== ============ =====


,normal_e,crit_e,normal_q,crit_q
0,6705.97,18193.31,0.0,0.0


In [15]:
chars[1].expect_dmg(ice_tree)

===== expected e dmg =====
normal e:    2125.19
crit e:      4890.07
===== ============ =====
===== expected q dmg =====
normal q:   13695.69
crit q:     31513.78
===== ============ =====


,normal_e,crit_e,normal_q,crit_q
0,2125.19,4890.07,13695.69,31513.78


# Draft after:

## Xiao

In [36]:
xiao_dmg_att = Damage_attribute(2536, 117, 66.4, 171.3, 0, 46.6+86.1)

In [37]:
xiao_dmg_att.show()

damage attribute: 
  atk:               2536
  elemental_mastery: 117
  crit_rate:         66.4
  crit_damage:       171.3
  cd_reduction:      0
  dmg_bonus:         132.7


In [38]:
xiao = Character('Xiao', 90, 'Anemo', xiao_dmg_att)

In [40]:
xiao_dmg = xiao.expect_dmg(3.49)

No monster type specified, using default monster level and resistance(93, 0.1)
===== expected dmg =====
normal: 9195.352785587467
crit:   24946.9921072988
===== ============ =====
